# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [21]:
# Datanya Saya Upload ke Github pribadi (Public).
# Jika ingin mengecek sumber kagglenya : https://www.kaggle.com/datasets/jeleeladekunlefijabi/ship-performance-clustering-dataset
# data=pd.read_csv("https://raw.githubusercontent.com/TokSeKa-uajy/datasetPython/refs/heads/main/dataSetMLPemula/df_normalisasi.csv") #Ini dataset yang sama, tapi yang masih di normalisasi sebelumnya di klustering.

dataMentah=pd.read_csv("https://raw.githubusercontent.com/TokSeKa-uajy/datasetPython/refs/heads/main/dataSetMLPemula/df_final.csv") # Ini dataset yang sudah reverse normalisasi dan encodingnya dan udah ada labelnya
dataMentah.head()

,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Operational_Cost_USD,Revenue_per_Voyage_USD,Efficiency_nm_per_kWh,Average_Load_Percentage,Draft_meters,Cargo_Weight_tons,Turnaround_Time_hours,Seasonal_Impact_Score,Weekly_Voyage_Count,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Weather_Condition,Cluster
0,12.597558,2062.983982,1030.943616,483832.354540,292183.273104,1.455179,93.769249,14.132284,1959.017882,25.867077,1.415653,1.0,Container Ship,Long-haul,Heavy Fuel Oil (HFO),Critical,Moderate,10
1,10.387580,1796.057415,1060.486382,483388.000509,883765.787360,0.290361,93.895372,14.653083,162.394712,63.248196,0.885648,6.0,Fish Carrier,Short-haul,Steam Turbine,Good,Rough,4
2,20.749747,1648.556685,658.874144,448543.404044,394018.746904,0.499595,96.218244,7.199261,178.040917,49.418150,1.405813,9.0,Container Ship,Long-haul,Diesel,Fair,Moderate,10
3,21.055102,915.261795,1126.822519,261349.605449,87551.375175,0.702906,66.193698,11.789063,1737.385346,22.409110,1.370704,1.0,Bulk Carrier,Transoceanic,Steam Turbine,Fair,Moderate,7
4,13.742777,1089.721803,1445.281159,287718.375160,676121.459632,1.331343,80.008581,9.727833,260.595103,64.158231,0.583383,8.0,Fish Carrier,Transoceanic,Diesel,Fair,Moderate,14


####Normalisasi dan Encoding Data
#####Pada tahap ini dilakukan persiapan akhir data sebelum klasifikasi:
- Fitur numerik dinormalisasi ke rentang [0, 1] menggunakan MinMaxScaler agar perhitungan jarak pada algoritma clustering tidak bias terhadap skala.
- Fitur kategorikal diubah menjadi format numerik dengan LabelEncoder, agar bisa diproses oleh model.
- Hasil transformasi disalin ke data sebagai dataset akhir yang siap digunakan.

In [22]:
fitur_kategorikal = [
    'Ship_Type',
    'Route_Type',
    'Engine_Type',
    'Maintenance_Status',
    'Weather_Condition'
]
fitur_numerik = [
    'Speed_Over_Ground_knots',
    'Engine_Power_kW',
    'Distance_Traveled_nm',
    'Operational_Cost_USD',
    'Revenue_per_Voyage_USD',
    'Efficiency_nm_per_kWh',
    'Average_Load_Percentage',
    'Draft_meters',
    'Cargo_Weight_tons',
    'Turnaround_Time_hours',
    'Seasonal_Impact_Score',
    'Weekly_Voyage_Count'
]

# 1. Normalisasi fitur numerik
scaler = MinMaxScaler()
df_scaled = dataMentah.copy()
df_scaled[fitur_numerik] = scaler.fit_transform(dataMentah[fitur_numerik])

# 2. Encoding fitur kategorikal
label_encoders = {}
for col in fitur_kategorikal:
    le = LabelEncoder()
    df_scaled[col] = le.fit_transform(df_scaled[col])
    label_encoders[col] = le  # Simpan encoder untuk inverse nanti

# 3. Hasil akhir
data = df_scaled.copy()

# Cek hasil
data.head()

,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Operational_Cost_USD,Revenue_per_Voyage_USD,Efficiency_nm_per_kWh,Average_Load_Percentage,Draft_meters,Cargo_Weight_tons,Turnaround_Time_hours,Seasonal_Impact_Score,Weekly_Voyage_Count,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Weather_Condition,Cluster
0,0.172666,0.625357,0.503367,0.967522,0.254676,0.968493,0.875361,0.913856,0.979420,0.230969,0.916364,0.000,1,1,1,0,1,10
1,0.025210,0.518488,0.518533,0.966615,0.877680,0.135914,0.877884,0.965983,0.057553,0.854481,0.385945,0.625,2,2,2,2,2,4
2,0.716607,0.459434,0.312357,0.895451,0.361920,0.285468,0.924353,0.219929,0.065581,0.623798,0.906517,1.000,1,1,0,1,1,10
3,0.736981,0.165847,0.552589,0.513144,0.039175,0.430789,0.323714,0.679323,0.865698,0.173291,0.871380,0.000,0,3,2,1,1,7
4,0.249079,0.235695,0.716076,0.566997,0.659007,0.879978,0.600080,0.473014,0.107941,0.869660,0.083444,0.875,2,3,0,1,1,14


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [23]:
# Pisahkan fitur (X) dan target (y)
X = data.drop(columns=['Cluster'])
y = data['Cluster']

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(2188, 17), y_train=(2188,)
Test set shape: X_test=(548, 17), y_test=(548,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [24]:
# Bagian 1: Pelatihan Model
# Definisikan setiap klasifikasi secara terpisah
knn = KNeighborsClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

print("Model training selesai.")

Model training selesai.


####Untuk membangun model klasifikasi berdasarkan label hasil clustering, digunakan lima algoritma populer dalam supervised learning, yaitu:

1. K Nearest Neighbors (KNN)
KNN adalah algoritma klasifikasi berbasis instance based learning, yang melakukan prediksi berdasarkan mayoritas label dari k tetangga terdekat dalam ruang fitur. KNN sangat sederhana dan efektif untuk data yang terpisah jelas antar kelas.

2. Decision Tree (DT)
Decision Tree bekerja dengan membagi data ke dalam cabang cabang berdasar fitur yang paling informatif, membentuk struktur seperti pohon. Algoritma ini mudah dipahami dan bekerja baik untuk dataset yang memiliki relasi kompleks antar fitur.

3. Random Forest (RF)
Random Forest adalah ensemble dari banyak Decision Tree yang dilatih pada subset data yang berbeda. Setiap pohon memberikan vote, dan hasil akhir ditentukan berdasarkan mayoritas. RF meningkatkan akurasi dan mengurangi risiko overfitting.

4. Support Vector Machine (SVM)
SVM mencoba menemukan hyperplane terbaik yang memisahkan kelas kelas dalam ruang fitur. Cocok untuk data berdimensi tinggi dan memberikan performa kuat meskipun ukuran data tidak terlalu besar.

5. Naive Bayes (NB)
Naive Bayes adalah algoritma probabilistik berdasarkan Teorema Bayes dengan asumsi independensi antar fitur. Meski sederhana, NB cukup efektif dan cepat untuk dataset berukuran kecil hingga sedang, terutama ketika fitur fitur bersifat kategorikal.

#####Alasan utama Saya menggunakan semua model, karena hampir semua yang Saya coba (kecuali KNN) memberikan hasil sempurna (akurasi 100%).


## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tabel berikut menampilkan hasil evaluasi 5 model klasifikasi berdasarkan akurasi dan F1-Score pada data testing. Ini menunjukkan seberapa baik model mampu mengenali pola klaster yang telah dibentuk sebelumnya:
- Semua model memiliki akurasi dan F1-Score tinggi, menunjukkan kualitas klasifikasi yang sangat baik.
- Nilai evaluasi pada data testing yang ditampilkan:
Test Accuracy, Test Precision, Test Recall, dan Test F1-Score.

In [25]:
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score
)

def evaluate_model(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    results = {
        'Train Accuracy': accuracy_score(y_train, y_train_pred),
        'Test Accuracy': accuracy_score(y_test, y_test_pred),
        'Test Precision': precision_score(y_test, y_test_pred, average='weighted'),
        'Test Recall': recall_score(y_test, y_test_pred, average='weighted'),
        'Test F1-Score': f1_score(y_test, y_test_pred, average='weighted'),
        'Confusion Matrix': confusion_matrix(y_test, y_test_pred)
    }
    return results



# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_train, y_train, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_train, y_train, X_test, y_test),
    'Random Forest (RF)': evaluate_model(rf, X_train, y_train, X_test, y_test),
    'Support Vector Machine (SVM)': evaluate_model(svm, X_train, y_train, X_test, y_test),
    'Naive Bayes (NB)': evaluate_model(nb, X_train, y_train, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Train Accuracy', 'Test Accuracy', 'Precision', 'Recall', 'F1-Score'])

rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Train Accuracy': metrics['Train Accuracy'],
        'Test Accuracy': metrics['Test Accuracy'],
        'Precision': metrics['Test Precision'],
        'Recall': metrics['Test Recall'],
        'F1-Score': metrics['Test F1-Score']
    })

summary_df = pd.DataFrame(rows)
print(summary_df)


                          Model  Train Accuracy  Test Accuracy  Precision  \
0     K-Nearest Neighbors (KNN)        0.988574       0.967153   0.970615   
1            Decision Tree (DT)        1.000000       1.000000   1.000000   
2            Random Forest (RF)        1.000000       1.000000   1.000000   
3  Support Vector Machine (SVM)        1.000000       1.000000   1.000000   
4              Naive Bayes (NB)        1.000000       1.000000   1.000000   

     Recall  F1-Score  
0  0.967153  0.967433  
1  1.000000  1.000000  
2  1.000000  1.000000  
3  1.000000  1.000000  
4  1.000000  1.000000  


## **Analisis Hasil Evaluasi Model Klasifikasi**

###Hasil evaluasi dari lima algoritma klasifikasi menunjukkan performa yang sangat tinggi, dengan nilai Accuracy, Precision, Recall, dan F1-Score semuanya mencapai 1.00 kecuali KNN yang sekitar 0.95++. Ini berarti model mampu mengklasifikasikan setiap data uji secara sempurna, tanpa kesalahan prediksi (Kecuali KNN, dengan sedikit kesalahan, tapi overall masih sangat baik).
- Tidak dilakukan proses tuning model karena performa awal sudah maksimal.
- Tidak ditemukan indikasi overfitting maupun underfitting dari hasil evaluasi di data uji.

####Catatan:
#####Berdasarkan hasil clustering yang sebelumnya menunjukkan separasi antar cluster yang sangat jelas (divalidasi melalui visualisasi PCA dan jarak centroid yang saling berjauhan), maka performa klasifikasi yang nyaris sempurna menjadi masuk akal. Hal ini mengindikasikan bahwa:
- Label hasil clustering cukup robust dan konsisten dengan pola dalam data.
- Model klasifikasi mampu merekonstruksi logika pembagian cluster dengan akurasi tinggi.

####Rekomendasi Lanjutan:
- Dataset saat ini memiliki karakteristik yang bersih dan sangat terstruktur, sehingga hasil evaluasi mungkin belum mencerminkan kondisi data yang lebih kompleks di lapangan (misalnya kondisi cuaca ekstrem, mesin tua, atau rute dinamis).
- Jika sistem ini digunakan untuk deployment di kasus nyata, disarankan menguji performa model pada data baru (misalnya rute atau tipe kapal yang belum pernah muncul sebelumnya) untuk menguji generalisasi.